# Configuracion de los volumenes logicos LVM

## CREANDO UN VOLÚMEN LÓGICO LVM EN 3 DISCOS

Debemos estar en la máquina virtual con los 5 discos sin formato

Ejecutar el siguiente comando para instalar las herramientas de soporte de lvm

`apt install lvm2`

Ejecutar `fdisk -l` para comprobar que los discos no tienen formato

***
```
root@debian12lvm1:~# fdisk -l
Disco /dev/sda: 10 GiB, 10737418240 bytes, 20971520 sectores
Modelo de disco: VBOX HARDDISK   
Unidades: sectores de 1 * 512 = 512 bytes
Tamaño de sector (lógico/físico): 512 bytes / 512 bytes
Tamaño de E/S (mínimo/óptimo): 512 bytes / 512 bytes
Tipo de etiqueta de disco: dos
Identificador del disco: 0xb4f2bd21

Disposit.  Inicio Comienzo    Final Sectores Tamaño Id Tipo
/dev/sda1  *          2048 18970623 18968576     9G 83 Linux
/dev/sda2         18972670 20969471  1996802   975M  5 Extendida
/dev/sda5         18972672 20969471  1996800   975M 82 Linux swap / Sola


Disco /dev/sdb: 5 GiB, 5368709120 bytes, 10485760 sectores
Modelo de disco: VBOX HARDDISK   
Unidades: sectores de 1 * 512 = 512 bytes
Tamaño de sector (lógico/físico): 512 bytes / 512 bytes
Tamaño de E/S (mínimo/óptimo): 512 bytes / 512 bytes


Disco /dev/sdd: 5 GiB, 5368709120 bytes, 10485760 sectores
Modelo de disco: VBOX HARDDISK   
Unidades: sectores de 1 * 512 = 512 bytes
Tamaño de sector (lógico/físico): 512 bytes / 512 bytes
Tamaño de E/S (mínimo/óptimo): 512 bytes / 512 bytes


Disco /dev/sde: 5 GiB, 5368709120 bytes, 10485760 sectores
Modelo de disco: VBOX HARDDISK   
Unidades: sectores de 1 * 512 = 512 bytes
Tamaño de sector (lógico/físico): 512 bytes / 512 bytes
Tamaño de E/S (mínimo/óptimo): 512 bytes / 512 bytes


Disco /dev/sdf: 5 GiB, 5368709120 bytes, 10485760 sectores
Modelo de disco: VBOX HARDDISK   
Unidades: sectores de 1 * 512 = 512 bytes
Tamaño de sector (lógico/físico): 512 bytes / 512 bytes
Tamaño de E/S (mínimo/óptimo): 512 bytes / 512 bytes


Disco /dev/sdc: 5 GiB, 5368709120 bytes, 10485760 sectores
Modelo de disco: VBOX HARDDISK   
Unidades: sectores de 1 * 512 = 512 bytes
Tamaño de sector (lógico/físico): 512 bytes / 512 bytes
Tamaño de E/S (mínimo/óptimo): 512 bytes / 512 bytes

```
***

o bien, para una forma un poco más compacta, ejecutar el comando 

`fdisk -l | grep sd`

en donde podemos observar de una forma más resumida la información de los dispositivos de almacenamiento que tenemos 

***
```
root@debian12lvm1:~# fdisk -l | grep sd
Disco /dev/sda: 10 GiB, 10737418240 bytes, 20971520 sectores
/dev/sda1  *          2048 18970623 18968576     9G 83 Linux
/dev/sda2         18972670 20969471  1996802   975M  5 Extendida
/dev/sda5         18972672 20969471  1996800   975M 82 Linux swap / Solaris
Disco /dev/sdb: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdd: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sde: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdf: 5 GiB, 5368709120 bytes, 10485760 sectores
Disco /dev/sdc: 5 GiB, 5368709120 bytes, 10485760 sectores

```
***

Ahora debemos asegurarnos que en el script del archivo **123** también se aplique para el dispositivo f, es decir, hay que descomentar las líneas especificadas para el dispositivo f

**123**

***
```
#!/bin/bash
#Estos comandos Establecen que van a ser discos msdos master boot recor>
parted /dev/sdb mklabel msdos
parted /dev/sdc mklabel msdos
parted /dev/sdd mklabel msdos
parted /dev/sde mklabel msdos
parted /dev/sdf mklabel msdos

#Estos comandos crean una particion primaria y le hacen un set a lvm
parted -s /dev/sdb unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sdc unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sdd unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sde unit mib mkpart primary 1 100% set 1 lvm on
parted -s /dev/sdf unit mib mkpart primary 1 100% set 1 lvm on

# Solo si es RHEL8
#Este comando le dice al kernel que reconozca esto nuevo
#udevadm settle

# De lo contrario ejecute
partprobe -s

```
***


Ejecutar el comando 
`sh 123`

***
```
root@debian12lvm1:~# sh 123
Information: You may need to update /etc/fstab.

Information: You may need to update /etc/fstab.

Information: You may need to update /etc/fstab.

Information: You may need to update /etc/fstab.

Information: You may need to update /etc/fstab.

/dev/sda: msdos partitions 1 2 <5>
/dev/sdb: msdos partitions 1
/dev/sdc: msdos partitions 1
/dev/sdd: msdos partitions 1
/dev/sde: msdos partitions 1
/dev/sdf: msdos partitions 1

```
***

y ahora ejecutar el comando, para verificar que los discos hayan sido particionados

`fdisk -l | grep sd`

***
```
root@debian12lvm1:~# fdisk -l | grep sd
Disco /dev/sda: 10 GiB, 10737418240 bytes, 20971520 sectores
/dev/sda1  *          2048 18970623 18968576     9G 83 Linux
/dev/sda2         18972670 20969471  1996802   975M  5 Extendida
/dev/sda5         18972672 20969471  1996800   975M 82 Linux swap / Solaris
Disco /dev/sdb: 5 GiB, 5368709120 bytes, 10485760 sectores
/dev/sdb1             2048 10485759 10483712     5G 8e Linux LVM
Disco /dev/sdd: 5 GiB, 5368709120 bytes, 10485760 sectores
/dev/sdd1             2048 10485759 10483712     5G 8e Linux LVM
Disco /dev/sde: 5 GiB, 5368709120 bytes, 10485760 sectores
/dev/sde1             2048 10485759 10483712     5G 8e Linux LVM
Disco /dev/sdf: 5 GiB, 5368709120 bytes, 10485760 sectores
/dev/sdf1             2048 10485759 10483712     5G 8e Linux LVM
Disco /dev/sdc: 5 GiB, 5368709120 bytes, 10485760 sectores
/dev/sdc1             2048 10485759 10483712     5G 8e Linux LVM
```
***

Ahora vamos a proceder a crear un volúmen físico, asi que debemos conocer el comando con la ayuda y el manual mediante los siguientes comandos

`pvcreate --help`
`man pvcreate`

Despues de leer la descripción y las opciones básicas del comando anterior podemos empezar a aplicarlo.

Para crear volúmenes físicos ejecutamos, lo que toma dos particiones para crearlo (el  tercer disco lo agregaremos posteriormente aunque podría ser agregado desde este comando agregando un tercer parámetro), recuerde que podemos usar una partición o el disco completo, pero como buena práctica vamos a utilizar la parcicion que usa todo el disco

`pvcreate /dev/sdb1 /dev/sdc1`

***
```
root@debian12lvm1:~# pvcreate /dev/sdb1 /dev/sdc1
  Physical volume "/dev/sdb1" successfully created.
  Physical volume "/dev/sdc1" successfully created.
```
***

Una vez que creamos el volumen físico entonces debemos crear el volumen de grupo, pero primero debemos conocer el comando que usaremos

`vgcreate --help`

Al pasarle un disco o partición que no es volúmen físico, el sistema lo hace autmáticamente. Las opciones que se le pasan son el nombre del grupo (en este caso es **myvg**) y los dispositivos o particiones asociadas

`vgcreate myvg /dev/sdb1 /dev/sdc1 /dev/sdd1`

***
```
root@debian12lvm1:~# vgcreate myvg /dev/sdb1 /dev/sdc1 /dev/sdd1
  Physical volume "/dev/sdd1" successfully created.
  Volume group "myvg" successfully created
```
***

Para el caso de **/dev/sdd1** no se encntraba creado, sin embargo, el comando vgcreate realizala creación del volumen físico de forma automática para poder crear el volumen de grupo.

Ahora mostramos los volúmenes de grupo con

`vgs`

La salida del comando es
***
```
root@debian12lvm1:~# vgs
  VG   #PV #LV #SN Attr   VSize   VFree  
  myvg   3   0   0 wz--n- <14.99g <14.99g
```
***

Se muestra un grupo de volúmenes, que corresponde a el volúmen que creamos.
De alli se lee que tiene tres volúmenes físicos (PV), no tiene volúmenes lógicos (LV), atributos (Attr), tamaño (VSize) y lo que tiene libre (VFree)

Ya está creado el volumen de grupo, ahora hay que crear el volumen lógico, en donde podemos usar todo el tamaño o solo una fracción del tamaño.


para crear el volumen hay que conocer el comando

`lvcreate --help`


Haciendo lo básico, especificando: el volumen de grupo de donde se tomará, se especifica el nombre y luego se especifica el tamaño (en algun momento analizaremos la diferencia entre los extends y los sizes o units (megas, gigas, teras, etc)), en este caso se crea el volumen lógico especificando 3 gigas (3G)

`lvcreate myvg -n mylv01 -L 3G`

***
```
root@debian12lvm1:~# lvcreate myvg -n mylv01 -L 3G
  Logical volume "mylv01" created.
```
***

Ahora comprobamos que se haya creado con 

`lvs`

***
```
root@debian12lvm1:~# lvs
  LV     VG   Attr       LSize Pool Origin Data%  Meta%  Move Log Cpy%Sync Convert
  mylv01 myvg -wi-a----- 3.00g 
```
***

  
El paso que sigue es crear el sistema de archivos

`mkfs.ext4 /dev/myvg/mylv01`


***
```
root@debian12lvm1:~# mkfs.ext4 /dev/myvg/mylv01
mke2fs 1.47.0 (5-Feb-2023)
Creating filesystem with 786432 4k blocks and 196608 inodes
Filesystem UUID: 6f8f316e-f699-4934-ad80-ab3b0f78f1fd
Superblock backups stored on blocks: 
	32768, 98304, 163840, 229376, 294912

Allocating group tables: done                            
Writing inode tables: done                            
Creating journal (16384 blocks): done
Writing superblocks and filesystem accounting information: done 
```
***

Lo que resta para terminar, es montar el volúmen lógico, (mediante fstab o montarlo temporalmente) procedemos a montar temporalmente

`mount /dev/myvg/mylv01 /mnt`


Ahora verificamos que se haya montado correctamente

`df -h /mnt/`

***
```
root@debian12lvm1:~# df -h /mnt/
S.ficheros              Tamaño Usados  Disp Uso% Montado en
/dev/mapper/myvg-mylv01   2.9G    24K  2.8G   1% /mnt

```
***

El tamaño no es exacto porque hay información que se va a los metadatos y ademas el sistema de archivos crea su estructura.
